In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.layers import *
from keras.models import Model
import keras

In [2]:
dataset = pd.read_pickle("./dataset.pkl")
dataset

,path,ageRange
13677,./UTKFace/1_1_0_20170109191253730.jpg.chip.jpg,0
1447,./UTKFace/1_1_2_20161219162225422.jpg.chip.jpg,0
10405,./UTKFace/1_0_2_20161219200203132.jpg.chip.jpg,0
11114,./UTKFace/1_0_1_20170110213647161.jpg.chip.jpg,0
12083,./UTKFace/1_0_0_20161219154724341.jpg.chip.jpg,0
...,...,...
16104,./UTKFace/90_1_0_20170120225317689.jpg.chip.jpg,90
8015,./UTKFace/92_1_0_20170110183501116.jpg.chip.jpg,90
23619,./UTKFace/90_1_0_20170120225510266.jpg.chip.jpg,90
13965,./UTKFace/95_1_0_20170117174948948.jpg.chip.jpg,90


In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

batch_size = 32
epochs = 50
input_shape = (256, 256, 3)

# 이미지 증강 세팅
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)



# 학습 데이터셋 생성
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

# 검증 데이터셋 생성
test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

Found 24000 validated image filenames belonging to 10 classes.
Found 6000 validated image filenames belonging to 10 classes.


<h1>전이 학습</h1>

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [6]:
# import requests

# url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# r = requests.get(url, allow_redirects=True)

# open('inception_v3.h5', 'wb').write(r.content)

In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(input_shape=input_shape, include_top=False, weights=None)
inception_model.load_weights('./inception_v3.h5')

inception_model.summary()

from tensorflow.keras.utils import plot_model

plot_model(inception_model, to_file='model.png', show_shapes=True, show_layer_names=True)

2023-04-26 17:49:05.360401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 17:49:05.392015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 17:49:05.392184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 17:49:05.392908: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [8]:
# 불러온 레이어는 학습 X
for i in inception_model.layers:
    i.trainable = False

# 특정 레이어만 학습
for i in inception_model.layers:
    if i.name == 'mixed6':
        i.trainable = True

In [9]:
# 원하는 레이어 가져오기
last_layer = inception_model.get_layer('mixed7')
print(last_layer)
print(last_layer.output)
print(last_layer.output_shape)

KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
(None, 14, 14, 768)


In [10]:
layer1 = tf.keras.layers.Flatten()(last_layer.output)
layer2 = tf.keras.layers.Dense(1024, activation="relu")(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
output_layer = tf.keras.layers.Dense(10, activation='softmax')(drop1)

model = tf.keras.Model(inception_model.input, output_layer)


In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델을 학습합니다.
checkpoint_path = "./transfer_best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [14]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    batch_size=batch_size,
    callbacks=[checkpoint]
)

Epoch 1/10
750/750 [==============================] - ETA: 0s - loss: 1.5967 - accuracy: 0.3620
Epoch 00001: val_accuracy improved from -inf to 0.40417, saving model to ./transfer_best_model.h5
750/750 [==============================] - 223s 297ms/step - loss: 1.5967 - accuracy: 0.3620 - val_loss: 1.3802 - val_accuracy: 0.4042
Epoch 2/10
750/750 [==============================] - ETA: 0s - loss: 1.4902 - accuracy: 0.3911
Epoch 00002: val_accuracy improved from 0.40417 to 0.45067, saving model to ./transfer_best_model.h5
750/750 [==============================] - 222s 296ms/step - loss: 1.4902 - accuracy: 0.3911 - val_loss: 1.3182 - val_accuracy: 0.4507
Epoch 3/10
750/750 [==============================] - ETA: 0s - loss: 1.4481 - accuracy: 0.4035
Epoch 00003: val_accuracy improved from 0.45067 to 0.46350, saving model to ./transfer_best_model.h5
750/750 [==============================] - 224s 299ms/step - loss: 1.4481 - accuracy: 0.4035 - val_loss: 1.2818 - val_accuracy: 0.4635
Epoch 4

KeyboardInterrupt: 

<h1>전이학습 테스트</h1>

In [23]:
models = {
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "DenseNet201": {"model":tf.keras.applications.DenseNet201, "perf":0},
    "EfficientNetB0": {"model":tf.keras.applications.EfficientNetB0, "perf":0},
    "EfficientNetB1": {"model":tf.keras.applications.EfficientNetB1, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "MobileNetV3Large": {"model":tf.keras.applications.MobileNetV3Large, "perf":0},
    "ResNet152V2": {"model":tf.keras.applications.ResNet152V2, "perf":0},
    "ResNet50": {"model":tf.keras.applications.ResNet50, "perf":0},
    "ResNet50V2": {"model":tf.keras.applications.ResNet50V2, "perf":0},
    "VGG19": {"model":tf.keras.applications.VGG19, "perf":0},
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
    "Xception": {"model":tf.keras.applications.Xception, "perf":0}
}

In [24]:
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':input_shape,
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False # 레이어를 동결 시켜서 훈련중 손실을 최소화 한다.
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [25]:
from time import perf_counter

# Train모델 학습
for name, model in models.items():
    
    # 전이 학습 모델 가져오기
    m = get_model(model['model'])
    models[name]['model'] = m
    
    start = perf_counter()
    
    # 모델 학습
    history = m.fit(
        train_generator,
        validation_data=test_generator,
        epochs=1,
    )
    
    # 학습시간과 val_accuracy 저장
    duration = perf_counter() - start
    duration = round(duration,2)
    models[name]['perf'] = duration
    print(f"{name:20} trained in {duration} sec")
    
    val_acc = history.history['val_accuracy']
    models[name]['val_acc'] = [round(v,4) for v in val_acc]

750/750 [==============================] - 233s 306ms/step - loss: 1.6512 - accuracy: 0.3470 - val_loss: 1.5143 - val_accuracy: 0.3893
DenseNet121          trained in 233.77 sec
750/750 [==============================] - 224s 296ms/step - loss: 1.6449 - accuracy: 0.3523 - val_loss: 1.5765 - val_accuracy: 0.3800
MobileNetV2          trained in 223.84 sec
750/750 [==============================] - 240s 311ms/step - loss: 1.5956 - accuracy: 0.3688 - val_loss: 1.5098 - val_accuracy: 0.3955
DenseNet201          trained in 240.35 sec
750/750 [==============================] - 226s 298ms/step - loss: 2.3039 - accuracy: 0.0986 - val_loss: 2.3029 - val_accuracy: 0.0953
EfficientNetB0       trained in 226.72 sec
750/750 [==============================] - 230s 301ms/step - loss: 2.3040 - accuracy: 0.1002 - val_loss: 2.3028 - val_accuracy: 0.0953
EfficientNetB1       trained in 230.6 sec
750/750 [==============================] - 227s 299ms/step - loss: 1.6601 - accuracy: 0.3413 - val_loss: 1.4978

In [33]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = './UTKFace/31_1_3_20170119200159884.jpg.chip.jpg'

img = image.load_img(img_path, target_size=(256, 256)) # 이미지 로드
img_array = image.img_to_array(img) # 넘파이 배열로 변환
img_array = np.expand_dims(img_array, axis=0) # 배치 차원 추가
img_array /= 255.

In [40]:
models

{'DenseNet121': {'model': <keras.engine.functional.Functional at 0x7facb5d7f250>,
  'perf': 233.77,
  'val_acc': [0.3893]},
 'MobileNetV2': {'model': <keras.engine.functional.Functional at 0x7faa50292250>,
  'perf': 223.84,
  'val_acc': [0.38]},
 'DenseNet201': {'model': <keras.engine.functional.Functional at 0x7faa51f57be0>,
  'perf': 240.35,
  'val_acc': [0.3955]},
 'EfficientNetB0': {'model': <keras.engine.functional.Functional at 0x7faa3b2e11f0>,
  'perf': 226.72,
  'val_acc': [0.0953]},
 'EfficientNetB1': {'model': <keras.engine.functional.Functional at 0x7faa384e4850>,
  'perf': 230.6,
  'val_acc': [0.0953]},
 'InceptionV3': {'model': <keras.engine.functional.Functional at 0x7faa324dcf40>,
  'perf': 226.88,
  'val_acc': [0.3877]},
 'MobileNetV3Large': {'model': <keras.engine.functional.Functional at 0x7faa30d15580>,
  'perf': 223.97,
  'val_acc': [0.0953]},
 'ResNet152V2': {'model': <keras.engine.functional.Functional at 0x7faa2bcb0940>,
  'perf': 245.97,
  'val_acc': [0.4268]},


In [43]:
models_result = []

for name, v in models.items():
    models_result.append([ name, models[name]['val_acc'][-1], 
                          models[name]['perf']])
    
df_results = pd.DataFrame(models_result, 
                          columns = ['model','val_accuracy','Training time (sec)'])
df_results.sort_values(by='val_accuracy', ascending=False, inplace=True)
df_results.reset_index(inplace=True,drop=True)
df_results

,model,val_accuracy,Training time (sec)
0,ResNet50V2,0.4322,225.87
1,ResNet152V2,0.4268,245.97
2,Xception,0.4050,229.00
3,DenseNet201,0.3955,240.35
4,DenseNet121,0.3893,233.77
5,InceptionV3,0.3877,226.88
6,MobileNetV2,0.3800,223.84
7,VGG16,0.3267,227.47
8,VGG19,0.3050,231.19
9,EfficientNetB0,0.0953,226.72
